# CKD Plan Designer Tutorial

## Install CKDPlanDesigner Package

In [26]:
! pip install -e project_code/

Obtaining file:///Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code
  Preparing metadata (setup.py) ... done
  Attempting uninstall: CKDPlanDesigner
    Found existing installation: CKDPlanDesigner 0.0.0
    Uninstalling CKDPlanDesigner-0.0.0:
      Successfully uninstalled CKDPlanDesigner-0.0.0
  Running setup.py develop for CKDPlanDesigner


## Load Data
- This tutorial uses the freely available synthea example dataset
    - Download it here: https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip
    - unzip it to SYNTHEA_PATH
 

In [27]:
import pandas as pd
import os

from CKDPlanDesigner.utils import data_preprocessing as pre

SYNTHEA_PATH = '/Users/nicholasdeveau/projects/bmi210/bmi210Project_data/synthea_data/'

#### Here we build our patient dataframe
- This contains a vecor for each patient containing the required information for plan designing 

In [28]:
patients_df = pre.preprocess_synthea_data(SYNTHEA_PATH)

/Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code/src/CKDPlanDesigner/utils/data_preprocessing.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_cond_df['VALUE'] = 1


In [29]:
patients_df

,patient_id,t2d,depression,bmi,hypertension,smoking,dob,race,ethnicity,gender,zip,eGFR,age
0,c8403116-d069-4130-aa02-39954fc971b5,True,False,False,False,False,1967-03-26,white,nonhispanic,F,NaN,66.9,55
1,73f124a4-e69e-4b8c-a447-e72d23dc54be,True,False,False,True,True,1967-06-14,white,nonhispanic,F,2170.0,70.5,55
2,31459cc4-45ed-4dee-b160-9f1a148a7724,False,False,True,False,False,1968-08-15,white,nonhispanic,M,NaN,75.4,54
3,799d0738-d5cd-4161-bdb8-e3c49e21ec66,False,False,True,True,True,1939-12-12,white,nonhispanic,M,2360.0,82.0,83
4,bae73dce-20fa-4a6a-8e46-b2ed541f9296,True,True,False,False,True,1958-04-27,white,nonhispanic,M,NaN,83.6,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,83c686df-0a1e-4950-9a2b-c84da98a2652,True,False,False,False,False,1962-01-24,asian,nonhispanic,F,NaN,81.7,60
112,9dfbf6ed-2378-4c27-9ff7-7e5c0619c236,True,False,True,False,False,1962-11-08,white,nonhispanic,F,NaN,88.6,60
113,1bacaab1-708b-42fe-8f12-fbe97a725440,True,False,True,False,False,1950-07-25,white,nonhispanic,F,2186.0,79.4,72
114,73fc00f8-a341-4f62-be27-07c35c3460bc,True,False,True,False,False,1941-05-04,white,nonhispanic,F,1027.0,84.8,81


## Working With a Patient

In [30]:
from CKDPlanDesigner.models import patients, plans, interventions
%load_ext autoreload
%autoreload patients
%autoreload plans
%autoreload interventions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
patient1_config = patients_df.to_dict(orient='index')[5]

In [32]:
def get_age(date_str):
    try:
        delta = datetime.datetime.now() - datetime.datetime.strptime(str(date_str), '%Y-%M-%d')
        return delta.days // 365
    except:
        return 55 # average age

In [33]:
get_age(patient1_config['dob'])

55

In [34]:
patients_df[(patients_df.age <= 74) & (patients_df.eGFR < 20)]

,patient_id,t2d,depression,bmi,hypertension,smoking,dob,race,ethnicity,gender,zip,eGFR,age
13,844d1c39-6ca2-48df-a3de-f9a12633f69c,True,False,False,False,False,1963-07-20,white,nonhispanic,F,NaN,8.9,59
64,af08878c-08dc-4938-aebd-cd5da315c479,True,False,True,False,False,1949-07-06,white,nonhispanic,M,NaN,5.6,73
78,5c06120a-9af5-4204-951b-7a8bfc465df3,True,False,True,True,False,1960-01-27,asian,nonhispanic,F,NaN,10.7,62
102,a96344e6-afe6-4eeb-9f95-00c62c152e8f,True,False,False,False,False,1982-01-14,black,nonhispanic,M,NaN,12.1,40


In [35]:
# patient_config = {'patient_id': 123,2
#                   'eGFR': 70,
#                   'depression': True
#                  }


# select the first patient and convert feature vector to a dictionary
# we pass this dictionary to the patient class on instantiation
patient1_config = patients_df.to_dict(orient='index')[13]
p1 = patients.Patient(**patient1_config)

patient2_config = patients_df.to_dict(orient='index')[44]
# patient2_config = patients_df.to_dict(orient='index')[15]
p2 = patients.Patient(**patient2_config)

#### Look at patient 1's charecteristics

In [36]:
p1.patient_config

{'patient_id': '844d1c39-6ca2-48df-a3de-f9a12633f69c',
 't2d': True,
 'depression': False,
 'bmi': False,
 'hypertension': False,
 'smoking': False,
 'dob': '1963-07-20',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'F',
 'zip': nan,
 'eGFR': 8.9,
 'age': 59}

In [37]:
# we automatically assign a CKD stage when a patient is created
p1.stage

### Generate a care plan

In [38]:
p1.generate_careplan()
print(p1.careplan.plan_name)

Plan: Smart Dialysis


In [13]:
p1.patient_config

{'patient_id': '844d1c39-6ca2-48df-a3de-f9a12633f69c',
 't2d': True,
 'depression': False,
 'bmi': False,
 'hypertension': False,
 'smoking': False,
 'dob': '1963-07-20',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'F',
 'zip': nan,
 'eGFR': 8.9,
 'age': 59}

In [39]:
p1.careplan

In [41]:
p1.careplan.list_components(show_savings=False)

Plan Components


['Dialysis', 'Education and Engagement', 'Nephro Engagement']

In [16]:
p1.careplan.all_components[1].desc_long

'Discuss desicion wrt. palliaive care vs. dialysis, emphasizing there is little evidence for outcomes benefit with dialysis at this stage.'

### Try it out for a different patient
- This patient has more co-morbidities. Check them out!
- Because of this, we should expect a more comprehensive care plan even though they're still just stage 2

In [17]:
import yaml

In [18]:
# yaml_path = os.path.join('/Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code/src/CKDPlanDesigner/configs/ix_config.yaml')
# ix_config_yaml = open(yaml_path)
# ix_config = yaml.load(ix_config_yaml, Loader=yaml.FullLoader)

In [19]:
p2.patient_config

{'patient_id': 'c9942738-49f3-4459-818f-8da20072ff76',
 't2d': True,
 'depression': True,
 'bmi': False,
 'hypertension': True,
 'smoking': True,
 'dob': '1935-10-27',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'M',
 'zip': 2474.0,
 'eGFR': 9.5,
 'age': 87}

In [20]:
p2.stage

In [21]:
p2.generate_careplan()
p2.careplan.list_components(show_savings=False)

Plan Components


['ComfortAndPain']

In [22]:
p2.patient_config

{'patient_id': 'c9942738-49f3-4459-818f-8da20072ff76',
 't2d': True,
 'depression': True,
 'bmi': False,
 'hypertension': True,
 'smoking': True,
 'dob': '1935-10-27',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'M',
 'zip': 2474.0,
 'eGFR': 9.5,
 'age': 87}

In [23]:
p2.patient_config

{'patient_id': 'c9942738-49f3-4459-818f-8da20072ff76',
 't2d': True,
 'depression': True,
 'bmi': False,
 'hypertension': True,
 'smoking': True,
 'dob': '1935-10-27',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'M',
 'zip': 2474.0,
 'eGFR': 9.5,
 'age': 87}

In [24]:
p2.careplan.list_components()

Plan Components


[('ComfortAndPain', 'Cost Reduction: 1')]

In [25]:
p2.careplan.list_components(show_savings=False)

Plan Components


['ComfortAndPain']

In [ ]:
"""
1) profile patient (patient_config)
2) classified into disease (patient.stage)
3) unoptimized charecteristics (trey creating this in the ontology)
    3a) unoptimmized = some min number of "bad" criteria (from patient_config)
"""

### Generate some Patients

In [76]:
def generate_px_info(idx):
    patient_config = patients_df.to_dict(orient='index')[idx]
    if patient_config['eGFR'] >=0 :
        px = patients.Patient(**patient_config)
        px.generate_careplan()
        plan_comps = px.careplan.list_components(show_savings=False)
        return (patient_config, plan_comps)
    else:
        return None, None

In [77]:
patients_df.to_dict(orient='index')[22]

{'patient_id': nan,
 't2d': True,
 'depression': True,
 'bmi': True,
 'hypertension': True,
 'smoking': True,
 'dob': nan,
 'race': nan,
 'ethnicity': nan,
 'gender': nan,
 'zip': nan,
 'eGFR': nan,
 'age': 55}

In [81]:
data_export = dict()
for i in range(0, 100, 4):
    config, comps = generate_px_info(i)
    if config:
        tmp = {'config': config, 'comps': comps}
        data_export[i] = tmp

Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components
Plan Components


In [83]:
import json

In [93]:
with open('patient_export_for_trey.json', 'w') as f:
    json.dump(json.dumps(data_export), f)

In [96]:
import json

with open('patient_export_for_trey.json', 'r') as f:
    tmp = json.loads(json.load(f))
    
tmp

In [97]:
tmp

{'0': {'config': {'patient_id': 'c8403116-d069-4130-aa02-39954fc971b5',
   't2d': True,
   'depression': False,
   'bmi': False,
   'hypertension': False,
   'smoking': False,
   'dob': '1967-03-26',
   'race': 'white',
   'ethnicity': 'nonhispanic',
   'gender': 'F',
   'zip': nan,
   'eGFR': 66.9,
   'age': 55},
  'comps': ['T2D Management']},
 '4': {'config': {'patient_id': 'bae73dce-20fa-4a6a-8e46-b2ed541f9296',
   't2d': True,
   'depression': True,
   'bmi': False,
   'hypertension': False,
   'smoking': True,
   'dob': '1958-04-27',
   'race': 'white',
   'ethnicity': 'nonhispanic',
   'gender': 'M',
   'zip': nan,
   'eGFR': 83.6,
   'age': 64},
  'comps': ['T2D Management', 'Depression Treatment']},
 '8': {'config': {'patient_id': 'd7f42940-d51d-4fc2-a604-03297e02f656',
   't2d': False,
   'depression': False,
   'bmi': True,
   'hypertension': False,
   'smoking': False,
   'dob': '1958-07-12',
   'race': 'white',
   'ethnicity': 'nonhispanic',
   'gender': 'M',
   'zip': nan